## Aprendizaje por Transferencia

Las redes neuronales tienen complicaciones para ser entrenadas de cero, pues requieren muchos datos y muchas GPUs. Esto es así en tanto que el lenguaje es muy complejo, y se requiere de una profunda base de datos para coger los matices. Entrenar un modelo de cero en NLP es muy costoso.

El **aprendizaje por transferencia** nos permite ahorrarnos parcialmente el coste de entrenamiento. La red neuronal se parte en dos objetos: el **cuerpo** y la **cabeza**. Las capas del cuerpo tienen unos pesos que entienden cómo funciona el lenguaje. Estos pesos pueden ser reutilizados para inicializar un nuevo modelo, con lo cual no hace falta que el nuevo modelo aprenda desde cero las características más básicas del lenguaje. La cabeza, por otro lado, contiene las capas que se focalizan en un problema en particular.

**Modelado del lenguaje**:
- Autorregresivo: Aprender cuál es la siguiente palabra dada una secuencia. My name --> is
- Enmascarado: Aprender una palabra "máscara" dada un contesto. My MASK name is ...

Con este concepto del modelado del lenguaje ya podemos entrenar un modelo: **pre-entrenamiento**. Entrenamos el modelo desde cero (muchos datos e.g. wikipedia, mucha computación). Después, con este *pretrained language model* (e.g. BERT) ya podemos hacer muchas tareas. 

**Fine-tuning**: Utilizamos los pesos del modelo pre-entrenado (cuerpo), y añadimos una cabeza específica para la tarea. Los pesos del cuerpo y sobre todo los de la cabeza (que son random en origen) son tuneados después usando *gradient descent*, dando pie a un *fine-tuned language model*.

<img src="aprendizaje_transferencia.png">

In [8]:
#Install hugging face libraries
!pip install transformers datasets

In [2]:
#Login to huggingface. You will need to sign up and create a token. Follow https://huggingface.co/

from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /Users/galogonzalvo/.huggingface/token


In [3]:
#Nos logamos a huggingface AI donde hay muchos datasets y modelos pre-entrenados
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Username: 

In [6]:
!apt install git-lfs
!git config --global user.email "galo.gonzalvo@gmail.com"
!git config --global user.name "Galo Gonzalvo"

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



In [10]:
#load dataset
from datasets import load_dataset

dataset = load_dataset("amazon_reviews_multi","es") #download amazon review data in spanish
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /Users/galogonzalvo/.cache/huggingface/datasets/amazon_reviews_multi/es/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

Tenemos un train, validation y test dataset

In [11]:
#Import packages

import random
import pandas as pd
from datasets import ClassLabel
from IPython.display import display, HTML

#Función para mostrar elementos aleatorios del dataset
def show_random_elements(dataset, num_examples=10):
    "Taken from https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb"
    
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"])

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0655494,product_es_0746693,reviewer_es_0751863,3,Bueno aceptable lo compre para hacer un champu natural y me va bien aunque no hace mucha espuma,BIEN,es,home
1,es_0746591,product_es_0377694,reviewer_es_0971770,4,"Todo muy bien, lo único que aunque actives las ventosas tienes que sujetarlo porque pierde presión y acaba cayendo, aún así cumple su función.",Funciona correctamente,es,home_improvement
2,es_0551590,product_es_0881435,reviewer_es_0251012,1,Si lo quieres utilizar sin conectarlo al cable de antena podras ver pocos canales ó ninguno.,Inhalambrico va regular,es,other
3,es_0907056,product_es_0010697,reviewer_es_0819996,1,"Leí en las preguntas que se realizan que valía para una bateria olympus omd em10 mark II, y no vale.",No vale o para olympus omd em10 II Mark,es,electronics
4,es_0192218,product_es_0742022,reviewer_es_0107865,4,"He tenido que comprar el producto dos veces. La primera me llegaron únicamente 4 piezas de las 7 anunciadas, además, dos de ellas rotas. Era un reacondicionado. Devolución y compro de nuevo. Esta vez han llegado todas, compré directamente SIN reacondicionado. Han llegado las 7 bien, aunque una de ellas tiene una ralla bastante grande, pero se puede utilizar.","Segunda compra, la primera mal estado",es,home
5,es_0206869,product_es_0867953,reviewer_es_0536938,5,"Esta bien, es lo que me esperaba, lo he usado para poner encima de una tarta.",Bien,es,toy
6,es_0535879,product_es_0527386,reviewer_es_0803740,1,No aguantan mucho en zonas humedas. Se me desprenden de la ducha en muy poco tiempo y he probado a ponerlos de varias formas. :(,Se desprenden con facilidad,es,home
7,es_0350497,product_es_0430493,reviewer_es_0941148,1,"el producto bien, pero me lo han cobrado dos veces",cobro duplicado,es,pet_products
8,es_0579291,product_es_0200458,reviewer_es_0295203,5,YO NO TENGO PERROS SINO GATOS PERO EL DE GATOS COSTABA BASTANTE MAS EN AQUEL MOMENTO DECORATIVO SIEMPRE Y CUANDO TENGAS OTRO MAS GRANDE PAA ALMACENAJE,MUY BONITO,es,home
9,es_0698172,product_es_0142466,reviewer_es_0054780,1,"No puedo opinar porque no he recibido el artículo ni lo voy a recibir, cuando compras a vendedores externos si hay algún problema pasan de todo ya es la segunda vez que me timan en Amazon.",Timo,es,pc


Vamos a construir un modelo de tal forma que si le pasamos un review, sea capaz de decirnos si es positivo o negativo

In [13]:
dataset.set_format("pandas") # es muy fácil convertir un dataset a pandas
df = dataset["train"][:] # seleccionas todos los rows
df.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0491108,product_es_0296024,reviewer_es_0999081,1,Nada bueno se me fue ka pantalla en menos de 8...,television Nevir,es,electronics
1,es_0869872,product_es_0922286,reviewer_es_0216771,1,"Horrible, nos tuvimos que comprar otro porque ...",Dinero tirado a la basura con esta compra,es,electronics
2,es_0811721,product_es_0474543,reviewer_es_0929213,1,Te obligan a comprar dos unidades y te llega s...,solo llega una unidad cuando te obligan a comp...,es,drugstore
3,es_0359921,product_es_0656090,reviewer_es_0224702,1,"No entro en descalificar al vendedor, solo pue...",PRODUCTO NO RECIBIDO.,es,wireless
4,es_0068940,product_es_0662544,reviewer_es_0224827,1,Llega tarde y co la talla equivocada,Devuelto,es,shoes


In [14]:
df['product_category'].value_counts()

home                        26962
wireless                    25886
toy                         13647
sports                      13189
pc                          11191
home_improvement            10879
electronics                 10385
beauty                       7337
automotive                   7143
kitchen                      6695
apparel                      5737
drugstore                    5513
book                         5264
furniture                    5229
baby_product                 4881
office_product               4771
lawn_and_garden              4237
other                        3937
pet_products                 3713
personal_care_appliances     3573
luggage                      3328
camera                       3029
shoes                        2754
digital_ebook_purchase       1843
video_games                  1733
jewelry                      1598
musical_instruments          1530
watch                        1490
industrial_supplies          1482
grocery       

In [15]:
df['stars'].value_counts()

5    40000
4    40000
3    40000
2    40000
1    40000
Name: stars, dtype: int64

In [16]:
dataset.reset_format() #volver al dataset object original si lo necesitas. muy fácil cambiar de formato con esta librería

In [ ]:
def filter_neutral_stars(examples):
    return examples["stars"] != 3

In [17]:
dataset = dataset.filter(lambda x: x["stars"] != 3) # no nos interesan los neutrales, queremos positivo o negativo. esta es la manera más eficiente de hacerlo

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [18]:
#Vamos a juntar las valoraciones en positiva o negativa
def merge_star_ratings(examples):
    if examples["stars"] <= 2:
        label = 0
    else:
        label = 1
    return {"labels": label}

In [19]:
dataset = dataset.map(merge_star_ratings)

  0%|          | 0/160000 [00:00<?, ?ex/s]

  0%|          | 0/4000 [00:00<?, ?ex/s]

  0%|          | 0/4000 [00:00<?, ?ex/s]

In [20]:
show_random_elements(dataset["train"], num_examples=3)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,labels
0,es_0712424,product_es_0694461,reviewer_es_0045033,2,"Para ser tan Caro no esperéis grandísima calidad, una cosa normal, estéticamente tampoco convence no se si sería por el color elegido, pero no queda como esperas es demasiada fina la correa. El pedido tardo más de lo esperado.",No lo volvería a comprar,es,wireless,0
1,es_0140956,product_es_0780994,reviewer_es_0609507,1,Este tamaño se me rompió dos veces. Sin embargo la de 5cms sigue viva después de utilizarla bastante.,Va mejor la perforadora de 5cm de diámetro,es,toy,0
2,es_0367959,product_es_0568071,reviewer_es_0806838,1,Poca Calidad. Una semana de uso y ya se están abriendo las costuras.,Poco recomendable si no pesa 50 kg,es,lawn_and_garden,0


#### Primero vamos a tokenizar las reseñas

In [21]:
from transformers import AutoTokenizer

model_checkpoint = "BSC-TeMU/roberta-base-bne" #Cargamos un modelo pre-entrenado de hugging face. Puedes mirar este modelo en https://huggingface.co/models
#Este se basa en un modelo fill-mask (modelado de lenguaje enmascarado) de "roberta", que es un modelo entrenado algo más poderoso que BERT
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/613 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/497k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [22]:
tokenizer.vocab_size # tamaño del vocabulario

50262

In [23]:
#tokenizamos y convertimos a número

text = "¡hola, me llamo lewis!"
tokenized_text = tokenizer.encode(text) 

for token in tokenized_text:
    print(token, tokenizer.decode([token])) #(id de palabra, palabra). 
    
#Lewis lo ha roto en 2 porque lewis no es común en español. Los tokens <s> </s> indican principio y final de frase

0 <s>
1465 ¡
12616 hola
66 ,
503  me
17111  llamo
532  le
19514 wis
55 !
2 </s>


In [24]:
encoded_text = tokenizer(text,return_tensors="pt") #el output es un tensor de pytorch
encoded_text 
#ids de cada token, attention mask es una manera de indicar al modelo qué partes de la frase debe aplicar atención o no.

{'input_ids': tensor([[    0,  1465, 12616,    66,   503, 17111,   532, 19514,    55,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [25]:
#Aplica el tokenizador a cada row en nuestro dataset
def tokenize_reviews(examples):
    return tokenizer(examples["review_body"], truncation=True) 
#Truncation sirve para truncar un texto exageradamente grande que exceda las dimensiones pre-establecidas de los vectores de nuestro modelo. En BERT, el vector tiene unas dimensiones máximas de 758

In [26]:
columns = dataset["train"].column_names
columns.remove("labels")
encoded_dataset = dataset.map(tokenize_reviews, batched=True, remove_columns=columns) #remove columns quita las columnas que no interesan para simplificar
encoded_dataset

  0%|          | 0/160 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 160000
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 4000
    })
})

In [27]:
encoded_dataset["train"][0]

{'labels': 0,
 'input_ids': [0,
  10626,
  3383,
  361,
  503,
  847,
  36181,
  4747,
  334,
  1111,
  313,
  1369,
  1635,
  342,
  403,
  1594,
  4162,
  2957,
  369,
  10925,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

#### Ya tenemos los textos preparados. Cargamos el modelo pre-entrenado

In [28]:
#Text classification es un caso particular de sequence classification.

from transformers import AutoModelForSequenceClassification

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at BSC-TeMU/roberta-base-bne were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.b

In [29]:
outputs = model(**encoded_text) #** stands for kwargs. keywoard arguments passed as a dictionary of arbitrary length
outputs
#el output son dos logits, uno para sentimiento positivo y otro para negativo

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0917, -0.1284]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

#### Definimos las métricas para evaluar el rendimiento

In [31]:
!pip install sklearn

     |████████████████████████████████| 7.8 MB 5.3 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=a4241dbc58916f533d4ff57ab6c639fc6fd4100e3658681607069d389b8fe77a
  Stored in directory: /Users/galogonzalvo/Library/Caches/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [32]:
from datasets import load_metric

metric = load_metric("accuracy")
metric

Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions: Predicted labels, as returned by a model.
    references: Ground truth labels.
    normalize: If False, return the number of correctly classified samples.
        Otherwise, return the fraction of correctly classified samples.
    sample_weight: Sample weights.
Returns:
    accuracy: Accuracy score.
Examples:

    >>> accuracy_metric = datasets.load_metric("accuracy")
    >>> results = accuracy_metric.compute(references=[0, 1], predictions=[0, 1])
    >>> print(results)
    {'accuracy': 1.0}
""", stored examples: 0)

In [ ]:
import numpy as np

#Construimos una función que nos devuelve las métricas a mitad de entrenamiento
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions,axis=1)
    return metric.compute(predictions=predictions, references=labels)

#### Pasamos al entrenamiento

In [34]:
#En transformers tenemos toda la lógica de entrenamiento en un lugar (paso a paso, cambiar los pesos, actualizar la logloss usando gradient descent, etc.)

from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]

batch_size = 16 #cuántos ejemplos vamos a pasar al modelo en cada iteración
num_train_epochs=2
num_train_samples = 20_000 #subset de los datos para que se entrene en 10min
train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(num_train_samples))
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=num_train_epochs,     
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch", 
    logging_steps=logging_steps,
    push_to_hub=True, #al final del training, con el push_to_hub, podemos poner lo aprendida encima para que el resto de la comunidad lo pueda usar
    push_to_hub_model_id=f"{model_name}-finetuned-amazon_reviews_multi"
)

/Users/galogonzalvo/opt/anaconda3/envs/tf/lib/python3.7/site-packages/transformers/training_args.py:951: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case ggonzalv/roberta-base-bne-finetuned-amazon_reviews_multi).
  FutureWarning,


In [35]:
#Necesitas git-lfs para esto. mejor en google colab

from transformers import Trainer

trainer = Trainer(
    model=model, 
    args=training_args, 
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer
)

trainer.train()

OSError: Looks like you do not have git-lfs installed, please install. You can install from https://git-lfs.github.com/. Then run `git lfs install` (you only have to do this once).

In [ ]:
#Subir nuestro modelo
trainer.push_to_hub() 

In [ ]:
#Para luego utilizar este modelo en otra aplicación

from transformers import pipeline

model_checkpoint = "lewtun/roberta-base-bne-finetuned-amazon_reviews_multi"
pipe = pipeline("sentiment-analysis", model=model_checkpoint)